In [1]:
import math
import os
import random
import struct as st

import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from itertools import compress
from numpy.random import RandomState

from sklearn import decomposition as dcmp

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [91]:
n_sounds = 49012
n_freq_bins = 1528
n_time_bins = 100

rng = RandomState(132) # arbitrary random state seed

var_explained_cutoff = 98 # target percent variance explained for PC space

In [4]:
data_folder = '../' # for Nate's MacBook Pro
#data_folder = '../../../data/' # for Chico

features_path = os.path.join(data_folder, 'acoustic_features_setA.h5')
labels_path = os.path.join(data_folder, 'label_names')
spectrograms_path = os.path.join(data_folder, 'sound_arrays.h5')

In [ ]:
def plot_components(dictionary, cmap):
    ''' Plots dictionary components (row vectors of data) as 2d images
    
    Note: assumes size of each component equals # frequency bins x # time bins in spectrogram dataset
    
    Parameters
    ----------
    dictionary: (numpy array, [# of spectrograms] x [# freq. bins x # time bins])
    
    
    '''
    # ADD CODE HERE

# Subsample Spectrogram Dataset

In [17]:
# Compile dict with indices of spectrograms organized by sound category
category_idxs = {}
with h5py.File(features_path) as features:
    labels = features['labels']
    
    for label_idx in range(19):
        # get list indices for which the given sound label/category applies (is stored as True)
        category_idxs[label_idx] = list(compress(range(n_sounds),labels[:,label_idx]))

In [58]:
# Get list of sound category names (strings)
category_names = []
with open(labels_path) as names:
    for line in names:
        category_names.append(line[:-1].strip())

In [71]:
# Visualization of how many sounds are in each category

n_per_category = [len(idxs) for idxs in category_idxs.values()] # number of sounds per category

%matplotlib qt
plt.bar(np.arange(19), n_per_category, tick_label=category_names)
plt.show()

for cat in range(19):
    print('%s:  %i' % (category_names[cat], len(category_idxs[cat])))

cat:  1104
dog:  763
fox:  563
hawk:  2477
owl:  15506
crow/raven:  656
goose:  0
duck:  1179
songbird:  4618
horse:  406
pig:  79
sheep:  94
music:  304
manmade:  608
speech:  304
ocean:  12268
rain:  2220
wind:  6233
fire:  2679


In [88]:
# Randomly (sub)sample up to 1000 sound samples from each category

max_samples = 1000 # maximum number of samples per category
train_prop = .8 # proportion of samples that are in training set (rest are in test set)
random.seed(13) # set random number generator at arbitrary seed

train_idxs = []
test_idxs = []
for idxs in category_idxs.values()
    if len(idxs) > max_samples:
        n_samples = max_samples
    else:
        n_samples = len(idxs)
    
    samples = random.sample(idxs, n_samples)
    train_idxs += samples[:round(train_prop*n_samples)]
    test_idxs += samples[round(train_prop*n_samples):]
        
train_idxs.sort()
test_idxs.sort()

In [ ]:
with h5py.File(spectrograms_path) as spectrograms:
    train_set = np.array(spectrograms['spectograms'][train_idxs,:,:]) # training set of spectrograms

# reshape to 2d array (rows = sounds, cols = features)
train_sets = train_set.reshape((len(train_idxs), n_freq_bins*n_time_bins))

# zero-average each feature across sounds in training set
features_avg = np.mean(spectrograms_subset,0)
train_set -= features_avg

# Perform PCA on Spectrograms

In [ ]:
# initialize and train PCA object
pca_model = dcmp.PCA(whiten = True, random_state = rng)
pca_model.fit(train_set)

In [ ]:
cum_var_explained = np.cumsum(pca_model.explained_variance_ratio_)

pct_var_explained = range(1,101,1)
n_pcs = [bisect.bisect(cum_var_explained, pct/100.)+1 for pct in pct_var_explained]

n_pcs_cutoff = bisect.bisect(cum_var_explained, var_explained_cutoff/100.)+1

% matplotlib inline
plt.plot(pct_var_explained, n_pcs, 'k')
plt.axhline(y = n_pcs_cutoff, color = 'r', linestyle = '--', alpha = .5)

plt.xlabel('% Variance Explained')
plt.ylabel('Number of PCs')

plt.show()

In [ ]:
print(n_pcs_cutoff)

In [ ]:
# Redo PCA with just enough PCs to hit the target variance explained
pca_model = dcmp.PCA(n_components = n_pcs_cutoff, whiten = True, random_state = rng)
pca_model.fit(train_set)

train_set_pc = pca_model_transform(train_set) # PC representation of data

# Train Sparse Coding Model

# Evaluate Dictionary Quality